使用PySpark在Vertex AI上构建一个零售数据推荐系统

### 目录

* [概述](#section-1)
* [数据集](#section-2)
* [目标](#section-3)
* [成本](#section-4)
* [创建启用组件网关和JupyterLab扩展的Dataproc集群](#section-5)
* [从笔记本连接到集群](#section-6)
* [探索数据](#section-7)
* [定义ALS模型](#section-8)
* [评估模型](#section-9)
* [将ALS模型保存到Cloud Storage](#section-10)
* [将推荐写入BigQuery](#section-11)
* [清理](#section-12)

## 概述
<a name="section-1"></a>

推荐系统是强大的工具，它们建模现有客户行为以生成推荐内容。这些模型通常构建复杂的矩阵，映射现有客户的偏好，以找到交叉兴趣并提供推荐内容。这些矩阵可能非常庞大，并且将受益于分布式计算和大内存池。在一个Vertex AI Workbench管理的笔记本实例中，您可以通过在一个Dataproc集群上使用PySpark处理数据来实现分布式计算。

*注意：这个笔记本文件是设计在一个使用Dataproc运行时生成的Python 3内核的[Vertex AI Workbench管理的笔记本](https://cloud.google.com/vertex-ai/docs/workbench/managed/create-instance)实例中运行的。这个笔记本的一些组件可能无法在其他笔记本环境中运行。*

## 数据集

本笔记本在BigQuery中使用`looker-private-demo.retail`数据集。可以通过在BigQuery中固定`looker-private-demo`项目来访问该数据集。在JupyterLab用户界面上，可以从Vertex AI Workbench托管笔记本实例执行此过程，而无需转到BigQuery用户界面。Vertex AI Workbench托管笔记本实例支持通过其BigQuery集成浏览BigQuery中的数据集和表。

在这个数据集中，将使用`retail.order_items`表来使用PySpark训练推荐系统。该表包含与数据集中的用户和商品相关的各种订单信息。

## 目标

本教程使用由Vertex AI Workbench托管笔记本实例提供的交互式PySpark功能，构建一个基于[协同过滤](https://en.wikipedia.org/wiki/Collaborative_filtering)方法的推荐模型。您将设置一个远程连接的Dataproc集群，并使用PySpark的MLlib库中实现的<a href="http://dl.acm.org/citation.cfm?id=1608614">交替最小二乘(ALS)</a>方法。

本笔记本中执行的步骤包括：

1. 将托管笔记本实例连接到具有PySpark的Dataproc集群。
2. 在笔记本中从BigQuery中探索数据集。
3. 预处理数据。
4. 在数据上训练一个PySpark ALS模型。
5. 评估ALS模型。
6. 生成推荐。
7. 使用PySpark-BigQuery连接器将推荐保存到BigQuery表中。
8. 将ALS模型保存到Cloud Storage存储桶。
9. 清理资源。

## 成本

本教程使用Google Cloud的以下计费组件：

* Vertex AI
* Dataproc
* BigQuery
* Cloud Storage

了解有关[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)、[Dataproc价格](https://cloud.google.com/dataproc/pricing)、[BigQuery价格](https://cloud.google.com/bigquery/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)的信息，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，将您的项目ID设置在这里。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

时间戳

如果您正在参加现场教程，您可能会使用共享测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个时间戳，并将其附加到在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

创建一个云存储桶

**无论您使用什么笔记本环境，以下步骤都是必需的。**

当您使用Cloud SDK提交训练作业时，您需要将包含训练代码的Python包上传到云存储桶中。Vertex AI会从这个包中运行代码。在本教程中，Vertex AI还会将您作业产生的训练模型保存在同一个存储桶中。使用这个模型工件，您可以创建Vertex AI模型和端点资源，以便提供在线预测。

在下方设置您的Cloud存储桶的名称。它必须在所有的Cloud存储桶中是唯一的。

您也可以更改`REGION`变量，它将在整个笔记本的其余部分中使用。确保[选择一个Vertex AI服务可用的地区](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。您不可以使用多区域存储桶来训练Vertex AI。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的桶不存在时才能运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

## 开始之前

ALS模型方法需要大量计算，可能需要很长时间才能在常规笔记本环境上训练，因此本教程使用带有PySpark环境的Dataproc集群。

### 创建具有组件网关启用和JupyterLab扩展的Dataproc集群
<a name="section-5"></a>

使用以下`gcloud`命令创建集群。

In [ ]:
CLUSTER_NAME = "[your-cluster-name]"
CLUSTER_REGION = "[your-cluster-region]"
CLUSTER_ZONE = "[your-cluster-zone]"
MACHINE_TYPE = "[your=machine-type]"

In [ ]:
! gcloud dataproc clusters create $CLUSTER_NAME \
--enable-component-gateway \
--region $CLUSTER_REGION \
--zone $CLUSTER_ZONE \
--single-node \
--master-machine-type $MACHINE_TYPE \
--master-boot-disk-size 100 \
--image-version 2.0-debian10 \
--optional-components JUPYTER \
--project $PROJECT_ID

另外，也可以通过Dataproc控制台创建集群。如果需要，可以在那里配置额外的设置，如网络配置和服务帐户。在配置集群时，请确保完成以下步骤：

- 为集群提供一个名称。
- 为集群选择一个区域和区域。
- 将集群类型选择为单节点。对于小型和概念验证用例，建议使用单节点集群。
- 启用组件网关。
- 在可选组件中，选择Jupyter Notebook。
- （可选）选择机器类型（最好选择高内存机器类型）。
- 创建集群。

## 从笔记本连接到集群
新的Dataproc集群运行时，对应的运行时会出现在笔记本中作为一个内核。创建的集群名称会出现在可以为此笔记本选择的内核列表中。在这个笔记本文件的右上角，点击当前的内核名称 **Python (local)**，然后选择在您的Dataproc集群上运行的Python 3内核。

注意以下内容：

- 您的Dataproc内核可能需要几分钟才会出现在内核列表中。
- 在本教程中的PySpark代码可以在Dataproc集群上的PySpark或Python 3内核上运行，但是为了运行将推荐保存到BigQuery表的可选代码，建议使用Python 3内核。

教程

## 探索数据
<a name="section-7"></a>

Vertex AI Workbench管理的笔记本实例使您可以使用BigQuery集成从管理的笔记本实例内部探索BigQuery内容。该功能使您可以查看表内容的元数据和预览，查询表，并获取表中数据的描述。

检查`STATUS`字段的分布。

#@bigquery
SELECT STATUS, COUNT(*) order_count FROM looker-private-demo.retail.order_items GROUP BY 1

#@bigquery
选择状态，COUNT(*) 订单数量 FROM looker-private-demo.retail.order_items 分组 BY 1

将`order_items`表与相同数据集中的`inventory_items`表连接，以检索订单的产品ID。

#@bigquery
以 user_prod_table 作为 (
选择 USER_ID, PRODUCT_ID, STATUS FROM looker-private-demo.retail.order_items AS a
加入
(SELECT ID, PRODUCT_ID FROM looker-private-demo.retail.inventory_items) AS b
在 a.inventory_item_id = b.ID )

选择 USER_ID, PRODUCT_ID, STATUS from user_prod_table

一旦在上述单元格中显示了来自BigQuery的结果，点击**查询并加载为DataFrame**按钮，并执行生成的代码存根以将数据获取到当前笔记本中作为一个数据帧。

*注意：默认情况下，数据将加载到`df`变量中，但如果需要的话，在执行单元格之前可以更改它。*

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
from google.cloud.bigquery import Client

client = Client()

query = """WITH user_prod_table AS (
SELECT USER_ID, PRODUCT_ID, STATUS FROM looker-private-demo.retail.order_items AS a
join
(SELECT ID, PRODUCT_ID FROM looker-private-demo.retail.inventory_items) AS b
on a.inventory_item_id = b.ID )

SELECT USER_ID, PRODUCT_ID, STATUS from user_prod_table"""
job = client.query(query)
df = job.to_dataframe()

### 数据预处理

要在现有数据上运行PySpark的ALS方法，必须有一些字段来量化`USER_ID`和`PRODUCT_ID`之间的关系，比如*用户给出的评分*。如果数据中已经存在这样的字段，它们可以被视为ALS模型的*显式反馈*。否则，表明关系的字段可以被视为*隐式反馈*。了解更多关于[PySpark的ALS方法的反馈信息](https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html#explicit-vs-implicit-feedback)。

在当前数据集中，由于没有这样的数值字段，`STATUS`字段被进一步用于量化`USER_ID`和`PRODUCT_ID`之间的关联。基于订单生命周期中它们发生的时间以及用户可能喜欢订单的可能性，`STATUS`字段被分配以下评分中的一个：

- 取消 - 1
- 退货 - 2
- 处理中 - 3
- 已发货 - 4
- 完成 - 5

给出的评分是主观的，可以根据使用情况进行修改。

In [ ]:
score_mapping = {
    "Cancelled": 1,
    "Returned": 2,
    "Processing": 3,
    "Shipped": 4,
    "Complete": 5,
}
df["RATING"] = df["STATUS"].map(score_mapping)

检查新生成的`RATING`字段的分布。

In [ ]:
df["RATING"].plot(kind="hist")

加载PySpark MLlib中所需的方法和类。

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession

使用配置了BigQuery-Spark连接器的Spark会话生成一个Spark会话。

*注意：如果笔记本连接到Dataproc集群，会话对象会显示`yarn`作为Master。*

In [ ]:
spark = (
    SparkSession.builder.appName("Recommendations")
    .config("spark.jars", "gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar")
    .getOrCreate()
)

spark

将pandas dataframe 转换为spark dataframe 以进行进一步处理。

In [ ]:
spark_df = spark.createDataFrame(df[["USER_ID", "PRODUCT_ID", "RATING"]])
spark_df.printSchema()
spark_df.show()

### 将数据分割成训练集和测试集。

In [ ]:
(train, test) = spark_df.randomSplit([0.8, 0.2], seed=36)
train.count(), test.count()

定义ALS模型
<a name="section-8"></a>

PySpark ALS推荐器，交替最小二乘法（Alternating Least Squares），是一种矩阵分解算法。其思想是构建一个将用户映射到行为的矩阵。这些行为可以是评论、购买、采取的各种选项等。由于矩阵的复杂性和规模，PySpark可以并行运行该算法。

ALS将尝试将评分矩阵R估计为两个较低秩矩阵X和Y的乘积。通常提到这些近似值被称为“因子”矩阵。在每次迭代中，其中一个因子矩阵保持不变，而另一个则使用最小二乘法求解。然后保持新解决的因子矩阵不变，同时解决另一个因子矩阵。

PySpark使用ALS因子分解算法的分块实现，将两组因子（称为“用户”和“产品”）分组为块，并通过仅在每次迭代中向每个产品块发送每个用户向量的一个副本，以及仅发送给需要该用户特征向量的产品块来减少通信。

基本上，与找到评分矩阵R的低秩近似值不同，这会找到偏好矩阵P的近似值，其中P的元素为1，如果r > 0则为1，如果r <= 0则为0。然后，这些评分作为与指示用户偏好强度相关的置信度值，而不是给定给项的明确评分。了解更多关于PySpark的ALS算法的信息。

In [ ]:
als = ALS(
    userCol="USER_ID",
    itemCol="PRODUCT_ID",
    ratingCol="RATING",
    nonnegative=True,
    implicitPrefs=False,
    coldStartStrategy="drop",
)

ALS 模型试图预测用户和物品之间的评分，因此可以使用 RMSE 来评估模型。

In [ ]:
evaluator = RegressionEvaluator(
    metricName="rmse", labelCol="RATING", predictionCol="prediction"
)

定义一个用于交叉验证的超参数网格。

In [ ]:
param_grid = (
    ParamGridBuilder()
    .addGrid(als.rank, [10, 50])
    .addGrid(als.regParam, [0.01, 0.1, 0.2])
    .build()
)
print("No. of settings to be tested: ", len(param_grid))

执行交叉验证并保存最佳模型。

In [ ]:
cv = CrossValidator(
    estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3
)
model = cv.fit(train)
best_model = model.bestModel

In [ ]:
print("##Parameters for the Best Model##")
print("Rank:", best_model._java_obj.parent().getRank())
print("MaxIter:", best_model._java_obj.parent().getMaxIter())
print("RegParam:", best_model._java_obj.parent().getRegParam())

评估模型
<a name="section-9"></a>
通过计算训练和测试数据上的RMSE评估模型。

In [ ]:
# View the rating predictions by the model on train and test sets
train_predictions = best_model.transform(train)
train_RMSE = evaluator.evaluate(train_predictions)

test_predictions = best_model.transform(test)
test_RMSE = evaluator.evaluate(test_predictions)

print("Train RMSE ", train_RMSE)
print("Test RMSE ", test_RMSE)

为所有用户生成推荐

可以使用ALS模型的`recommendForAllUsers()`方法生成用户所需数量的推荐。

In [ ]:
# Generate 10 product recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

为特定用户生成推荐

前面的步骤已经为`nrecommendations`数据框中的所有用户生成并存储了指定数量的产品推荐。要获取单个用户的推荐，可以查询这个数据框对象。

In [ ]:
# get product recommendations for the selected user (USER_ID = 1)
nrecommendations.where(nrecommendations.USER_ID == 1).select(
    "recommendations.PRODUCT_ID", "recommendations.rating"
).collect()

## 将ALS模型保存到云存储（可选）
<a name="section-10"></a>

PySpark的`ALS.save()`方法会在指定路径创建一个文件夹，并在其中保存训练好的模型。托管笔记本实例的环境中提供了云存储文件浏览器，您可以使用它将模型保存到云存储桶中。

使用ALS对象的` .save（）`函数将模型写入Cloud Storage存储桶。

In [ ]:
# Save the trained model
GCS_MODEL_PATH = "gs://" + BUCKET_NAME + "/recommender_systems/"
best_model.save(GCS_MODEL_PATH + "rcmd_model")

## 将推荐写入BigQuery（可选）
<a name="section-11"></a>

为了向最终用户或任何应用程序提供推荐，可以使用Spark的BigQuery连接器将`recommendForAllUsers()`方法的输出保存到BigQuery表中。

### 在BigQuery中创建数据集

以下代码单元格在BigQuery中创建一个新数据集。

#@bigquery
-- 在BigQuery中创建一个数据集
CREATE SCHEMA recommender_sys
OPTIONS(
  location="us"
  )

### 写入 BigQuery 推荐

PySpark的 BigQuery 连接器需要两个必要字段：一个 *BigQuery 表名称* 和一个 *Cloud 存储路径来写入临时文件* 在保存模型时。这两个字段可以在写入到 BigQuery 推荐时提供。

In [ ]:
DATASET = "[your-dataset-name]"
TABLE = "[your-bigquery-table-name]"
GCS_TEMP_PATH = "[your-cloud-storage-path]"

nrecommendations.write.format("bigquery").option(
    "table", "{}.{}".format(DATASET, TABLE)
).option("temporaryGcsBucket", GCS_TEMP_PATH).mode("overwrite").save()

清理
<a name="section-12"></a>
要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在本教程中创建的各个资源：

In [ ]:
# remove the BigQuery dataset created for storing the recommendations and all of its tables
! bq rm -r -f -d $PROJECT:$DATASET

# remove the Cloud Storage bucket created and all of its tables
! gsutil rm -r gs://$BUCKET_NAME

In [ ]:
# delete the created Dataproc cluster
! gcloud dataproc clusters delete $CLUSTER_NAME --region=$CLUSTER_REGION